# Data Cleaning
**Kaggle Competition:** House Prices: Advanced Regression Techniques

Saloni Sharma
***
In this notebook, I will clean the data set by replacing null values as well as removing a few entire columns. This entire notebook will be run twice, once for the training set and once for the test data set.

In [1]:
# import necessary library
import pandas as pd

## 1. View data

In [2]:
# Load data set
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")

In [3]:
print("There are", len(train), "rows in the data set.")

There are 1460 rows in the data set.


In [4]:
# View column names and data types
train.info()
# Also, note that there are some columns with fewer non-null values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
# Count of null values in each column
train.isnull().sum().sort_values(ascending=False).head(20)
# There are 19 columns with null values in the data set

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
Utilities          0
dtype: int64

## 2. Null/NA Values
Looking at the description of the data shows that null values don't necessarily indicate that the data is missing or unknown, but rather, their meaning is often that the feature is absent from or non-existent for that property. For example: 

| Feature | Value | Meaning |
| --- | --- | --- |
| PoolQC | NA | No Pool | 
| MiscFeature | NA | No Misc. Feature |
| Alley | NA | No Alley Access | 
| Fence | NA | No Fence | 

Features where NA/NaN actually means null (missing data):
- LotFrontage 
- GarageYrBlt
- MasVnrArea
- Electrical

#### 2.a. Replace null values with correct meaning.

In [6]:
# the features where null means 'absent'
features_to_fix = ["PoolQC", "MiscFeature", "Alley", "FireplaceQu", "Fence",
                "GarageCond", "GarageType", "GarageFinish", "GarageQual",
                "BsmtExposure", "BsmtFinType2", "BsmtFinType1", "BsmtCond", "BsmtQual",
               ]

In [7]:
# confirm that they are object types
train[features_to_fix].dtypes

PoolQC          object
MiscFeature     object
Alley           object
FireplaceQu     object
Fence           object
GarageCond      object
GarageType      object
GarageFinish    object
GarageQual      object
BsmtExposure    object
BsmtFinType2    object
BsmtFinType1    object
BsmtCond        object
BsmtQual        object
dtype: object

In [8]:
# replace null in these features with 'absent'
train[features_to_fix] = train[features_to_fix].fillna("Absent")

In [9]:
# confirm that there no more nulls
train[features_to_fix].isnull().sum()

PoolQC          0
MiscFeature     0
Alley           0
FireplaceQu     0
Fence           0
GarageCond      0
GarageType      0
GarageFinish    0
GarageQual      0
BsmtExposure    0
BsmtFinType2    0
BsmtFinType1    0
BsmtCond        0
BsmtQual        0
dtype: int64

#### 2.b. Replace actual null data with average of the feature.

In [10]:
# print null counts
print("The number of nulls in each column:\n",
    train[['LotFrontage', 'GarageYrBlt', 'MasVnrType', 'MasVnrArea', 'Electrical']].isnull().sum())
# print data types
print("\nThe data types are:\n",
    train[['LotFrontage', 'GarageYrBlt', 'MasVnrType', 'MasVnrArea', 'Electrical']].dtypes)

The number of nulls in each column:
 LotFrontage    259
GarageYrBlt     81
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64

The data types are:
 LotFrontage    float64
GarageYrBlt    float64
MasVnrType      object
MasVnrArea     float64
Electrical      object
dtype: object


In [11]:
# LotFrontage
# Replace null values by the average for that neighborhood
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.mean()))

In [12]:
# GarageYrBlt
# Replace year by average of years from neighborhood
train['GarageYrBlt'] = train.groupby('Neighborhood')['GarageYrBlt'].transform(
    lambda x: x.fillna(x.mean().astype(int)))

In [13]:
# MasVnrType 
# This is a little different from the other features because 
# it has both 'None' and NaN values

# I will replace the missing null values with the mode,
# meaning the most frequently used type in the neighborhood
train['MasVnrType'] = train.groupby('Neighborhood')['MasVnrType'].transform(
    lambda x: x.fillna(x.mode()[0]))

In [14]:
# MasVnrArea
# All of these nulls are for properties with no masonry veneer is not used
print(train[['MasVnrArea', 'MasVnrType']].loc[train.MasVnrArea.isnull()])

# The data mostly uses zero for area of properties without masonry veneer
print("\nThe properties without Masonry Veneer use:")
print(train[['MasVnrArea', 'MasVnrType']].loc[train.MasVnrType=="None"].value_counts())

# So, the corresponding areas can be set to zero
train['MasVnrArea'].fillna(0, inplace=True)

      MasVnrArea MasVnrType
234          NaN       None
529          NaN       None
650          NaN       None
936          NaN       None
973          NaN       None
977          NaN       None
1243         NaN      Stone
1278         NaN       None

The properties without Masonry Veneer use:
MasVnrArea  MasVnrType
0.0         None          859
1.0         None            2
344.0       None            1
312.0       None            1
288.0       None            1
dtype: int64


In [15]:
# Electrical
# There is only one row where this is null [1379]
# I will replace it with the most frequently used system in its neighborhood
train['Electrical'] = train.groupby('Neighborhood')['Electrical'].transform(
    lambda x: x.fillna(x.mode()[0]))

In [16]:
# confirm there are no more nulls left
print("The number of nulls:")
print(train[['LotFrontage', 'GarageYrBlt', 'MasVnrType', 'MasVnrArea', 'Electrical']].isnull().sum())

The number of nulls:
LotFrontage    0
GarageYrBlt    0
MasVnrType     0
MasVnrArea     0
Electrical     0
dtype: int64


For the entire data set, there are no more null values remaining.

In [17]:
train.isnull().sum().sort_values(ascending=False).head()

SalePrice      0
Heating        0
RoofStyle      0
RoofMatl       0
Exterior1st    0
dtype: int64

## 3. Unnecessary Features
There are some columns with almost all values being the same, so they will not be a useful feature to use for prediction, and I will remove those.

#### 3.a. Remove columns with few categories where most values are the same.
I got this idea from this notebook: https://www.kaggle.com/fedi1996/house-prices-data-cleaning-viz-and-modeling

In [18]:
# Check for columns with few categories
for col in train.columns:
    if len(train[col].value_counts()) < 3:
        print(train[col].value_counts(), "\n")

Pave    1454
Grvl       6
Name: Street, dtype: int64 

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64 

Y    1365
N      95
Name: CentralAir, dtype: int64 



In [19]:
# I will drop columns for 'Street' and 'Utilities'
# They both have mostly the same values for all properties
train.drop(columns=['Street', 'Utilities'], inplace=True)

## 4. Test Data Nulls
Here, I will replace the null values that are in the test data, but not in the train data.

In [20]:
# load updated test data
test = pd.read_csv("CleanTest.csv")

In [21]:
# view remaining nulls
test.isnull().sum().sort_values(ascending=False).head(15)

MSZoning        4
Functional      2
BsmtHalfBath    2
BsmtFullBath    2
BsmtFinSF2      1
KitchenQual     1
Exterior2nd     1
GarageCars      1
GarageArea      1
Exterior1st     1
SaleType        1
BsmtFinSF1      1
TotalBsmtSF     1
BsmtUnfSF       1
BsmtQual        0
dtype: int64

In [22]:
# column names for features with nulls in test data
nullcols = (test.isnull().sum().sort_values(ascending=False).head(14).index)

In [23]:
test[nullcols].dtypes

MSZoning         object
Functional       object
BsmtHalfBath    float64
BsmtFullBath    float64
BsmtFinSF2      float64
KitchenQual      object
Exterior2nd      object
GarageCars      float64
GarageArea      float64
Exterior1st      object
SaleType         object
BsmtFinSF1      float64
TotalBsmtSF     float64
BsmtUnfSF       float64
dtype: object

In [24]:
for col in nullcols:
    if test[col].dtype==object:
        # replace with most frequent value from neighborhood
        test[col] = test.groupby('Neighborhood')[col].transform(
            lambda x: x.fillna(x.mode()[0]))
    else:
        # replace with average for neighborhood
        test[col] = test.groupby('Neighborhood')[col].transform(
            lambda x: x.fillna(x.mean()))

In [25]:
# confirm there are no more nulls in the test data
test.isnull().sum().sort_values(ascending=False).head()

SaleCondition    0
BsmtQual         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
dtype: int64

## 5. Save Data
Since the data is now cleaned, save the data set to a new .csv file.

In [26]:
train.to_csv("CleanTrain.csv", index=False)

In [27]:
test.to_csv("CleanTest.csv", index=False)